<a id="top"></a>
# Long Period TESS Targets with Kepler Counterparts
***


## Prerequisites
- Python syntax
- Something about exoplanet transits, link to free resource

## Learning Goals

By the end of this tutorial, you will:

- 

To use as a guideline:
"Set a minimum threshold for orbital period (should be long-period orbits, greater than 6mos). Find all TESS 20-seconds targets that have data for these Kepler TCEs to create an even longer baseline."

## Introduction
The [Kepler initial and follow-on (K2) missions confirmed over 2,600 exoplanets](https://exoplanets.nasa.gov/keplerscience/) and contributed to an unprecedented exposion of knowledge about planetary systems in our galaxy. In addition to confirmed exoplanets, Kepler identified thousands more potential planetary candidates, referred to as Kepler Objects on Interest (KOI's). 

The Transiting Exoplanet Survey Satellite (TESS) began its observations just before the completion of the K2 mission; there may be long-period (greater than 6 months) exoplanets whose transits were observed by both missions. By combinining observations from both missions, we could potentially view multiple transits and confirm the existence of new exoplanets. 

Possible transits observed by Kepler are called Threshold Crossing Events, or TCEs. More specifically, according to the [NASA Exoplanet Archive documentation](https://exoplanetarchive.ipac.caltech.edu/docs/Kepler_TCE_docs.html), 

>"A Threshold-Crossing Event (TCE) is a sequence of transit-like features in the flux time series of a given target that resembles the signature of a transiting planet to a sufficient degree that the target is passed on for further analysis."

The workflow for this notebook consists of:
* [Main Content](#Main-Content)
    * [Loading Data](#Loading-Data)
    * [File and Data Information](#File-and-Data-Information)
* [Visualization](#Visualization)
* [Exercises](#Exercises)
* [Additional Resources](#Additional-Resources)

## Imports
Describe the main packages we're using here and their use-case for this notebook. If there's something unusual, explain what the library is, and why we need it.
- *numpy* to handle array functions
- *astropy.io fits* for accessing FITS files
- *astropy.table Table* for creating tidy tables of the data
- *matplotlib.pyplot* for plotting data

In [60]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from astropy.io import fits
from astropy.table import Table
from astroquery.mast import Mast
from astroquery.mast import Observations

# Added by JRK
import requests
from astropy.io import ascii
import lightkurve as lk

***

## Load full table of Kepler Threshold Crossing Events, or TCEs and filter by period

https://exoplanetarchive.ipac.caltech.edu/docs/program_interfaces.html#tce
```

In [44]:
# Load the TCE table as an astropy table.
tab=Table.read('https://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?table=q1_q17_dr25_tce',format='ascii.csv')
len(tab)

34032

In [47]:
# View the full list of column names
# More info here: https://exoplanetarchive.ipac.caltech.edu/docs/API_tce_columns.html 
tab.columns

<TableColumns names=('kepid','tce_plnt_num','tce_rogue_flag','tce_period','tce_period_err','tce_time0bk','tce_time0bk_err','tce_impact','tce_impact_err','tce_duration','tce_duration_err','tce_depth','tce_depth_err','tce_model_snr','tce_prad','tce_prad_err','tce_eqt','tce_eqt_err','tce_insol','tce_insol_err','tce_steff','tce_steff_err','tce_slogg','tce_slogg_err','tce_sradius','tce_sradius_err')>

The NASA Exoplanet Archive provides detailed information about the [Data Columns in the Kepler TCE Table](https://exoplanetarchive.ipac.caltech.edu/docs/API_tce_columns.html). Some of the most important columns for us will be:
- kepid, Kepler identification number, which we can use when searching the MAST Archive.
- tce_period, the orbital period in days.

JRK - DR24 has an "autovetter" parameter that could help us identify likely planet candidates instead of false positives.

In [51]:
# Only include those table rows with TCE period greater than 180 days
tab=tab[tab['tce_period']>180]
len(tab)

11157

In [52]:
tab.pprint()

 kepid   tce_plnt_num tce_rogue_flag ... tce_sradius tce_sradius_err
-------- ------------ -------------- ... ----------- ---------------
 2304168            2              0 ...       3.332           1.718
 2303102           10              0 ...       1.257           0.352
  892376            1              0 ...       0.605           0.074
  892376            4              0 ...       0.605           0.074
  893507            2              0 ...       1.684           0.939
  893507            1              1 ...       1.684           0.939
  893507            3              1 ...       1.684           0.939
  893507            4              0 ...       1.684           0.939
  893507            5              0 ...       1.684           0.939
  893507            7              0 ...       1.684           0.939
     ...          ...            ... ...         ...             ...
12116009            1              0 ...       0.437           0.056
11757328            1             

In [71]:
str(tab['kepid'][0])

'2304168'

## Find all TESS 20-second targets for these Kepler TCEs

In [61]:
obs_table = Observations.query_object('KIC 2304168')
obs_table.pprint()

intentType obs_collection provenance_name ...   obsid        distance     
---------- -------------- --------------- ... --------- ------------------
   science           TESS            SPOC ...  27463635                0.0
   science           TESS            SPOC ...  62431369                0.0
   science           TESS            SPOC ...  62870781                0.0
   science           TESS            SPOC ...  92616912                0.0
   science           TESS            SPOC ...  95133363                0.0
   science           TESS            SPOC ...  62349772 304.60998167941784
   science           TESS            SPOC ...  62793221 304.60998167941784
   science           TESS            SPOC ...  62367644 359.16608247249354
   science           TESS            SPOC ...  62367570 359.16608247249354
   science           TESS            SPOC ...  62810479 359.16608247249354
       ...            ...             ... ...       ...                ...
   science           HLSP

In [62]:
obs_table['target_name']

TESS FFI
TESS FFI
TESS FFI
TESS FFI
TESS FFI
137316027
137316027
1876944736
1876944736
1876944736
1876944736


### Lightkurve?

In [69]:
search_result=lk.search_lightcurve('')
search_result

SearchResult containing 0 data products.

In [ ]:
# Can specify author='Kepler' and author='TESS'
none=True
while none:
    for i in np.arange(0,100):
        search_result=lk.search_lightcurve(str(tab['kepid'][i]),author='TESS')
        if len(search_result)>1:
            none=False

No data found for target "2304168".
No data found for target "2303102".
No data found for target "892376".
No data found for target "893507".
No data found for target "893647".
No data found for target "1025986".
No data found for target "1026133".
No data found for target "1026474".
No data found for target "1028246".
No data found for target "1162339".
No data found for target "1294756".
No data found for target "1294869".
No data found for target "1297272".
No data found for target "1431114".
No data found for target "1431501".
No data found for target "1433399".
No data found for target "1434660".
No data found for target "1435448".
No data found for target "1434976".
No data found for target "1570924".
No data found for target "1572201".
No data found for target "1572114".
No data found for target "1573138".
No data found for target "1573094".
No data found for target "1576144".
No data found for target "1717528".
No data found for target "1720673".
No data found for target "17193

Where possible (if the code supports it), use code examples that visually display the data in the tutorial. For example, if you are showing an object such as a Table, display a preview of the table:

In [ ]:
yourProd[0:5]

In [ ]:
# Download the products
output = Observations.download_products(yourProd, mrp_only=False, cache=False)
output

Explain pertinent details about the file you've just downloaded. For example, if working with Kepler light curves, explain what's in the different file extensions:

```
- No. 0 (Primary): This HDU contains metadata related to the entire file.
- No. 1 (Light curve): This HDU contains a binary table that holds data like
  flux measurements and times. We will extract information from here when we
  define the parameters for the light curve plot.
- No. 2 (Aperture): This HDU contains the image extension with data collected
  from the aperture. We will also use this to display a bitmask plot that
  visually represents the optimal aperture used to create the SAP_FLUX column in
  HDU1.

```

In [ ]:
file = output['Local Path'][0]
print('The HDU list of our output file:\n')
print(fits.info(file))

data = fits.getdata(file, 1)['FLUX']
time = fits.getdata(file, 1)['TIME']

## Visualization

When presenting any visuals and/or plots from the data, make sure you are using color palettes that are color-blind friendly and using language that keeps accessibility in mind. The most common form of color vision deficiency involves differentiating between red and green, so avoiding colormaps with both red and green will avoid many problems in general. Use descriptive keywords not pertaining to the color of the object you are referring to. It is good practice to make your plots and images large enough to ensure that important details are not hard to see. On the same note, make sure that tick labels, legends, and other plot notations are not too small, and make sure they are descriptive enough that the user can understand what is being represented by the data. 

Let's plot the first four images of the Kepler TPF we just downloaded to see where the center of the PSF is located...

In [ ]:
imgs = 4

fig, axs = plt.subplots(1, imgs, figsize=(20, 20))

for idx in range(0, imgs):
    # Plotting
    axs[idx].imshow(data[idx], cmap='bone', origin='lower')
    
    # Formatting
    axs[idx].set_title(f'Image #{idx}', fontsize=25)
    axs[idx].tick_params(axis='both', which='major', labelsize=20)

Looks like it's typically located around (x,y)=(4,4). Let's gather all the images and extract the flux at (4,4) from each of them to patch our lightcurve together...

In [ ]:
lightcurve = []

for idx in range(0, len(data)):
    
    flux = data[idx][4, 4]
    lightcurve.append(flux)

In [ ]:
# Plotting
plt.figure(1, figsize=(10, 6))
plt.plot(time, lightcurve)

# Formatting
obj = fits.getheader(file)['OBJECT']
plt.title(f'Object: {obj}', fontsize=25)
plt.xlabel('Time - 2454833 (BKJD days)', fontsize=20)
plt.ylabel('Flux (e-/s)', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=15)

## Exercises
Exercises are optional, but encouraged. Exercises can be woven into the main content of your tutorial, or appear in their own section toward the end of the tutorial. Final exercises can be more challenging, similar to homework problems. They can be minimal or take as long as 30 minutes to an hour to complete. If you do have one or more exercises in your tutorial, be sure to leave a blank code cell underneath each to show the reader that they're meant to try out their new skill right there. You may also want to include a "solutions" notebook next to your main notebook for the reader to check their work after they have finished their attempt.

## Additional Resources

This section is optional. Try to weave resource links into the main content of your tutorial so that they are falling in line with the context of your writing. For resources that do not fit cleanly into your narrative, you may include an additional resources section at the end of your tutorial. Usually a list of links using Markdown bullet list plus link format is appropriate:

- [MAST API](https://mast.stsci.edu/api/v0/index.html)
- [Kepler Archive Page (MAST)](https://archive.stsci.edu/kepler/)
- [Kepler Archive Manual](https://archive.stsci.edu/kepler/manuals/archive_manual.pdf)
- [Exo.MAST website](https://exo.mast.stsci.edu/)

## Citations
Provide your reader with guidelines on how to cite open source software and other resources in their own published work.

```
If you use `astropy` or `lightkurve` for published research, please cite the
authors. Follow these links for more information about citing `astropy` and
`lightkurve`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `lightkurve`](http://docs.lightkurve.org/about/citing.html)

```

## About this Notebook
Let the world know who the author of this great tutorial is! If possible and appropriate, include a contact email address for users who might need support (for example, `archive@stsci.edu`). You can also optionally include keywords, your funding source, or a last update date in this section.

**Author(s):** Jessie Blogs, Jenny V. Medina, Thomas Dutkiewicz <br>
**Keyword(s):** Tutorial, lightkurve, kepler, introduction, template <br>
**Last Updated:** Aug 2022 <br>
**Next Review:** Mar 2023
***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 